<a href="https://colab.research.google.com/github/Behnaz81/MachineLearningDaily/blob/main/day12_prompt_engineering_intro/MachineLearning12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prompt Engineering - session 1

***Prompt engineering*** is the process of discovering prompts that yield desired results.

A ***prompt*** is the input you provide, typically text, when interfacing with an AI model like ChatGPT.

You can use a very simple prompt and get a _good enough_ response, but when you plan to put this prompt into production, ypu should work on the prompt! Because then mistakes cost you money. There are some points you should pay attention to:

1. _Vague direction_

  You should specify everything that can help the AI model to give you a better result.

2. _Unformatted output_

  If you want to use the output in an API you may need it in JSON format and specifying the format can help you not have different formats everytime you send the same prompt.

3. _Missing examples_

  If you give the model an example of a good answer, it can help to get a better output. For example if you want it to suggest a good name for a channel, you can give it some examples of names that you like.

4. _Limited evaluation_

  It's better to define a rating system to measure if the answer is good or bad.

5. _No task division_

  It's better to specify the steps. not just asking the model to do something in one prompt.

## Five Principles of Prompting

### 1. Give Direction

You can use a series of prompts to get a better result. For example first ask for tips on naming a channel and then ask the model to generate the name considering the rules he suggested.

Another way is to give an external resource in the prompt and ask the model to use the principles suggested. This may cost more money but may be worth it!

In image generation you can name an artist to give the model direction.

Note that giving too much direction can lead to generate an image that isn't consistent with all of your criteria. In these cases you should choose which element is more important.



---


### 2. Specify Format

When putting AI tools into production, it's more important to specify the output format.

For text generation models, it an be helpful to use the JSON format. It makes it simpler to parse and spot errors and to render the front-end HTML of an application.

YAML is another popular choice because it has a parseable and human-readable structure.

Using JSON as the final format can help you use it in an application and also check if there's an error in the formatting using a JSON parser.

For image generation the format can be different, like `stock photo`, `illustration`, `oil painting`, `in Minecraft` and so on.

Sometimes the first and second principles may overlap. When there are clashes between style and format, it's often best to drop whichever element is less important for your final result.

---

### 3. Provide Examples

A prompt with no examples is ***zero-shot***. If a model does something zero-shot it shows that it's a powerful model. If you provide one example it's a ***one-shot*** and if you provide multiple examples it's called ***few-shot***. A paper shows that adding one example with a prompt can improve accuracy from 10% to near 50%!

It's sometimes easier to give examples than to say what it's that you like about them.

The more examples you provide, and the lesser the diversity between them, the less the creativiy. Try to include 3 examples at most.

There is some evidence that direction works better than providing examples. So better to try the Give Direction principle first.

In image generation an example is a base image. This is called *img2img* in the open source Stable Diffusion community. Different base image with the same prompt can lead to a totally different result.

---

### 4. Evaluate Quality

If you just do some trials and have no feedback loop to judge the quality of your response, it's called ***blind prompting***. This is fine for general use but when you want to use a prompt multiple times or build a production application that relies on a prompt, you should consider measuring the results.

***Evals*** are a standardized set of questions with predefined answers to test model performance.

Sometimes more sophisticated models are used to evaluate less advanced models.

To evaluate and compare results of two prompts you can make a simple rating system. Run every prompt for n times and save the results in a spreadsheet. Then use a rating system, display the results randomly without especifying the prompt and rate the results. This can be tedius to rate all the responses manually and your ratings might not represent the preference of your audience. But it's better than nothing! You can use this to decide if you need to privide examples in your prompt or not.

Another way is ***ground truth***, refrencing answers to test cases to programmatically rate the results. Reasons why you should evaluate the results programmatically:

1. *Cost*: Prompts with too many tokens or using an expensive model aren't practical for production use.

2. *Latency*: The longer the prompt or the larger the model the longer it takes to answer. This can harm user experience.

3. *Calls*: Many AI systems require multiple calls in a loop to complete a task that can slow down the process.

4. *Performance*: Use a physics engine or other models to predict real world results.

5. *Classification*: Determine how often a prompt correctly labels given text, using another AI model or rule-based labeling.

6. *Reasoning*: Check which instances the AI fails to apply logical reasoning or gets the math wrong versus reference cases.

7. *Hallucinations*: How frequently you prompt results to invention new terms not included in the prompt's context.

8. *Safety*: Use a safety filter or detection system to flag scenarios where the system might return unsafe results.

9. *Refusals*: How often the system incorrectly refuses to fulfill a reasonable user request by flagging known refusal language.

10. *Adversarial*: Prevent ***prompt injection*** that can get the model to run undesirable prompts.

11. *Similarity*: Use shared words or phrases or vectore distance to measure similarity between the result and refrence text.

In image generation to evaluate the prompt you can try different combination of prompts and see the results.

---

### Divide Labor

Use task decomposition to break problems into their component parts. For example, first ask for a list of names for a channel then ask the model to rate the names out of 10. You can do the rating in some steps too. For example first examine clarity then memorability and finally take a score from human interaction as well. You can use the phrase `Let's think step by step` to do what OpenAI calls giving the model time to think.

Using an AI model to generate a prompt for an AI model is ***meta prompting***.

Early AI products under the hood chain multiple prompts together. This is called ***AI chaining***.



## Practical Examples

In [15]:
!pip install dotenv

In [16]:
import os
import openai
from dotenv import load_dotenv

We use environmental variables to save API key.

In [18]:
load_dotenv('.env')
API_KEY = os.getenv("OPENAI_API_KEY")
client = openai.OpenAI(api_key=API_KEY)

**model**
- ID of model to use.

**messages**
- A list of messages that determine the context of conversation so far.

**temperature**
- Between 0 and 2.
- Randomness of answer. The closer the value to 0 the more deterministic the answer and the closer it is to 2 the more creative the model will be.

**max_tokens**
- Maximum number of tokens of the answer.

In [19]:
response = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[
        {'role': 'system', 'content': 'You are a helpful assistant'},
        {'role': 'user', 'content': 'Which NHL team plays in Pittsburgh?'}
    ],
    temperature=0.5,
    max_tokens=1024
)

There are 3 choices for `role`: assistant, user and system.

`role: system` is used only once, at the beginning. I helps us set up the tone and the context for the assistant future responses.

You can ask for multiple `choices` in response using n parameter.

`finish_reason` can have many values. It indicates why answering stopped. It can be beacause of reaching max tokens limit or a natural stop point.

You can also see the number of tokens in `usage` part. This can help you calculate how much it costs you to run you program. You can also check your OpenAI dashboard to see this information.

In [20]:
response

ChatCompletion(id='chatcmpl-Bw63F6qQCxuLJbin8xcigKsoOOyj0', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The NHL team that plays in Pittsburgh is the Pittsburgh Penguins.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1753185985, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=12, prompt_tokens=23, total_tokens=35, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [21]:
response.choices[0].message.content

'The NHL team that plays in Pittsburgh is the Pittsburgh Penguins.'